# Ecommerce Product Analysis

Download product reivews.

Summarize benefits, features, and emotions



---

### About Alton

Follow me for more data and tutorials

- twitter: https://twitter.com/alton_lex @alton_lex

- linkedin: https://www.linkedin.com/in/altonalexander/


### About Data Winners

Join the conversation:

- private Discord community

- Video tutorials

- Feedback and support on this and other scripts

Join now: https://datawinners.gumroad.com/l/data-analytics-for-seo

# Import Libraries

In [98]:
!pip install bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


# Find one product on amazon

Including product and reviews

In [ ]:
# inputs

# my asin
asins1 = "B0859BZ69K"

product_title = "Toddler Balance Bike"

In [12]:

# Scrape reviews for primary ASIN and store in a dataframe
primary_url = f"https://www.amazon.com/product-reviews/{asins1}/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews"
headers={'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Mobile Safari/537.36'}


response= requests.get(primary_url, headers=headers, timeout=10)
soup = BeautifulSoup(response.text, 'html.parser')


Scraping reviews from https://www.amazon.com/product-reviews/B0859BZ69K/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews
Index([], dtype='object')


In [74]:

reviews = []
dedup = set()


for review in soup.find_all('div', {'class' : 'a-section'}):
    


        author = review.find('span', {'class': 'a-profile-name'})
        if author:
            author = author.string.strip()
        else:
            author = None
        title = review.find('span', {'class': 'review-title-content'})
        if title:
            title = title.text.strip()
        else:
            title = None
        #date = review.find('span', {'class': 'a-size-base a-color-secondary'}).string.strip()[3:]
        content = review.find('span', {'class': 'review-text-sub-contents'})
        if content:
            content = content.text.strip()
        else:
            content = None
        date = review.find('span', {'class': 'review-date'})
        if date:
            date = " ".join(date.text.strip().split(" ")[-3:])
        else:
            date = None

        rating = review.find('span', {'class': 'a-icon-alt'})
        if rating:
            rating = rating.string.strip()[0]
        else:
            rating = None

        if content and content not in dedup:
            review_dict = {'Author': author, 'Title': title, 'Date': date, 'Content': content, 'Rating': rating}
            reviews.append(review_dict)
            dedup.add( content )

reviews

[{'Author': 'Cynthia DeVries',
  'Title': 'Bike for two year old boy for balance',
  'Date': 'February 24, 2023',
  'Content': 'Perfect way for a 2 year old to learn to balance on a bike before riding one with the pedals and chains. Smart idea. Our Grandson Love’s 💕 it.',
  'Rating': '5'},
 {'Author': 'Morgan Backus',
  'Title': 'Perfect for my 2 year old!',
  'Date': 'February 7, 2023',
  'Content': 'We have bought balance bikes before. This one is by far my favorite. I love that it has a handle in the middle for easy carry and my 2 year old can hop on and just go! We take it everywhere with us.',
  'Rating': '5'},
 {'Author': 'Kindle Customer',
  'Title': 'Grandson loves it',
  'Date': 'February 21, 2023',
  'Content': 'Easy to put together and grandson loves it',
  'Rating': '5'},
 {'Author': 'N',
  'Title': 'Perfect',
  'Date': 'February 9, 2023',
  'Content': 'Cute bike for the price. Plus, It only took 5 minutes to assemble by myself. Wheels light up!',
  'Rating': '5'},
 {'Autho

In [75]:
primary_df = pd.DataFrame(reviews)
primary_df

,Author,Title,Date,Content,Rating
0,Cynthia DeVries,Bike for two year old boy for balance,"February 24, 2023",Perfect way for a 2 year old to learn to balan...,5
1,Morgan Backus,Perfect for my 2 year old!,"February 7, 2023",We have bought balance bikes before. This one ...,5
2,Kindle Customer,Grandson loves it,"February 21, 2023",Easy to put together and grandson loves it,5
3,N,Perfect,"February 9, 2023","Cute bike for the price. Plus, It only took 5 ...",5
4,Allison Vargas,Perfect,"February 9, 2023",Totally perfect for my 2 year old. So happy wi...,5
5,Mac R,Awesome Little Bike,"January 25, 2023",I’ve purchased this for my 2.5 year old nephew...,5
6,Sterwana,Great product,"February 1, 2023",Easy to assemble,5
7,G. Leatherwood,Super sturdy,"January 19, 2023",A great first bikeThe kids never needed traini...,5
8,Grace,Great balance bike,"December 28, 2022",My son is almost 3 and this is perfect for him...,4
9,EveryDayUser,So close to perfection,"November 29, 2022","The bike is REALLY good for the price, and pro...",5


# Create a list of bullet points

In [77]:
# install libraries for openai
%pip install openai

# load libraries
import openai
from pprint import pprint

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [78]:
# Set your api key
# copy and paste your open api key as a varible

openai.api_key = "sk-PASTEYOURKEYHERE"

In [92]:
def textgen_for_datawinners(
        prompt,
        model = "text-davinci-003",
        max_length = 2000
    ):
    """ A generic wrapper for the endpoint """
    
    response = openai.Completion.create(
      model=model,
      prompt=prompt,
      temperature=0.7,
      max_tokens=max_length,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    return response.get('choices',[])[0]['text']

In [93]:
reviews_list_string = ""

for i, row in primary_df.iterrows():
    
    reviews_list_string = reviews_list_string + "\n\n- " +row['Content']

In [94]:
prompt = """Product: """+product_title+"""
    Please analyze the following product reviews. List the top 5 benefits, the top 5 features, and top 5 emotional benefits consumers experience when using this product.""" + reviews_list_string
print(prompt)

Product: Toddler Balance Bike
    Please analyze the following product reviews. List the top 5 benefits, the top 5 features, and top 5 emotional benefits consumers experience when using this product.

- Perfect way for a 2 year old to learn to balance on a bike before riding one with the pedals and chains. Smart idea. Our Grandson Love’s 💕 it.

- We have bought balance bikes before. This one is by far my favorite. I love that it has a handle in the middle for easy carry and my 2 year old can hop on and just go! We take it everywhere with us.

- Easy to put together and grandson loves it

- Cute bike for the price. Plus, It only took 5 minutes to assemble by myself. Wheels light up!

- Totally perfect for my 2 year old. So happy with it!

- I’ve purchased this for my 2.5 year old nephew. The size was perfect and it was easy to assemble. The quality is great too! He loves it

- Easy to assemble

- A great first bikeThe kids never needed training wheels when they upgraded to a pedal bike


In [97]:
response = textgen_for_datawinners(prompt)
print(response)



Top 5 Benefits:
1. Allows kids to learn to balance before riding with pedals and chains.
2. Lightweight and easy to carry.
3. Quick and easy assembly.
4. Wheels light up.
5. Perfect size for 2-2.5 year olds.

Top 5 Features:
1. Handle in the middle for easy carry.
2. Lightweight and easy to move.
3. Wheels light up.
4. Adjustable bars and seat.
5. High quality and good price.

Top 5 Emotional Benefits:
1. Grandson loves it.
2. Kids never need training wheels.
3. Perfect for small kiddos.
4. Great first bike.
5. Cute bike.
